# Relative velocities in DustPy

In [ ]:
import sys

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import cmocean

import dustpy as dp
import dustpy.constants as c

if '..' not in sys.path:
    sys.path.append('..')

import helper

In [ ]:
s = dp.Simulation()

s.ini.grid.rmin = 0.1 * c.au
s.ini.grid.rmax = 400 * c.au
s.ini.grid.Nr = 300

s.ini.grid.mmax *= 1e3
s.ini.grid.mmin /= 1e3
s.ini.grid.Nmbpd = 15

s.ini.gas.alpha = 1e-3

s.ini.gas.SigmaRc = 1e6 * c.au
s.ini.gas.Mdisk = 141.417 * c.M_sun # that's made to get 200 g/cm^2 at 1 au

s.initialize()

s.gas.boundary.outer.setcondition('const_val')
s.gas.boundary.outer.setboundary()

s.gas.T = 200 * (s.grid.r / c.au)**0.5
s.gas.T.updater = None

s.update()

Check Temperature and Surface density values

In [ ]:
np.interp(c.au, s.grid.r, s.gas.T), np.interp(c.au, s.grid.r, s.gas.Sigma)

In [ ]:
keys = {
    'rad': 'Radial Motion',
    'azi': 'Azimuthal Drift',
    'vert': 'Vertical Settling',
    'brown': 'Brownian Motion',
    'turb': 'Turbulent Motion',
    'tot': 'Total'}

In [ ]:
ir = np.abs(s.grid.r - c.au).argmin()
a = s.dust.a[ir]

f, axs = plt.subplots(2, 3, figsize=(12, 8), sharex=True, sharey=True, gridspec_kw={'hspace':0.1, 'wspace':0.1})

vmin = 1e-2
vmax = 1e4
norm = LogNorm(vmin=vmin, vmax=vmax, clip=False)
levels = np.geomspace(vmin, vmax, 3 * int(np.log10(vmax) - np.log10(vmin)) + 1)

white_lines = [100, 1000]

for i, (key, lab) in enumerate(keys.items()):
    ax = axs.ravel()[i]
    
    vrel = getattr(s.dust.v.rel, key)[ir, ...]
    cc = ax.contourf(a, a, vrel + 1e-100, levels, cmap='cmo.algae', norm=norm, extend='both')
    
    # style each plot
    ax.set_aspect('equal')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(keys[key])
    ax.minorticks_off()
    
    # add white contour lines
    wc = ax.contour(a, a, vrel + 1e-100, white_lines, colors='w', linestyles=['--', '-'])
    if i == 0:
        manual = [(1e-3, 1e0), (1e-3, 1e2)]
    else:
        manual = False
    ax.clabel(wc, fmt=lambda val: f'{val/100:.0f} m/s', manual=manual)


# color bar

pos1, pos0 = [ax.get_position() for ax in axs.ravel()[[2, -1]]]
w = pos0.width / 10
cax = f.add_axes([pos0.x1 + w, pos0.y0, w, pos1.y1 - pos0.y0])
cb = f.colorbar(cc, cax=cax)
cb.set_label('relative velocity [cm / s]')
cb.set_ticks(10.**np.arange(np.log10(vmin), np.log10(vmax)+1))


# axes ticks and labels

for ax in axs[1,:]:
    ax.set_xlabel('particle size [cm]')
for ax in axs[:, 0]:
    ax.set_ylabel('particle size [cm]')
    
for ax in axs.ravel():
    ax.set_xticks(10.**np.arange(-4, 3))
    ax.set_yticks(10.**np.arange(-4, 3))

In [ ]:
f.savefig(helper.output_dir / 'rel_vel.pdf', transparent=True, bbox_inches='tight')

## Approximate Turbulent Velocities

In [ ]:
Re = (s.gas.alpha * s.gas.Sigma * c.sigma_H2 / (2 * s.gas.mu))
rhos = s.dust.rhos.mean(-1)
ya = 1.6
a_12 = Re**-0.5 / ya * 2 * s.gas.Sigma / (np.pi * rhos)
a_BT = (8 * s.gas.Sigma / (np.pi * rhos) * Re**-0.25 *
        np.sqrt(s.gas.mu / (3 * np.pi * s.gas.alpha)) * (4 * np.pi / 3 * rhos)**-0.5)**0.4

u_g = s.gas.cs * np.sqrt(3 * s.gas.alpha / 2)
St = s.dust.St
St0 = St[:, [0]]

In [ ]:
eps = a[None, :] / (4 * a_12[:, None]) - a_12[:, None] / 4

In [ ]:
dv_0 = np.zeros_like(s.dust.Sigma)

# small grains
_dv = Re[:, None]**0.25 * (St - St0)
mask = a[None, :] > a_12[:, None]
dv_0[mask] = _dv[mask]

# medium sizes
_dv = (1 - eps) * Re[:, None]**0.25 * (St - St0) + eps * (St - St0) * np.sqrt(3 * St)
mask = (a[None, :] < a_12[:, None]) & (a[None, :] < 5 * a_12[:, None])
dv_0[mask] = _dv[mask]

# large sizes
_dv = np.sqrt(3 * St)
mask = a[None, :] > 5 * a_12[:, None]
dv_0[mask] = _dv[mask]

dv_0 *= u_g[:, None]

In [ ]:
dv_1 = np.sqrt(2 / 3) * dv_0.copy()
dv_1[a[None, :] < a_12[:, None]] = 0.0

In [ ]:
def a2St(a):
    #return a * rhos[ir] * np.pi / (2 * s.gas.Sigma[ir])
    return np.interp(a, s.dust.a[ir, :], s.dust.St[ir, :])

def St2a(St):
    #return St / (rhos[ir] * np.pi / (2 * s.gas.Sigma[ir]))
    return np.interp(St, s.dust.St[ir, :], s.dust.a[ir, :])

\begin{align}
v_\mathrm{mon} = c_\mathrm{s} \, \sqrt{\frac{3}{2}\alpha} \,
    \begin{cases}
        Re^{1/4} \left(\mathrm{St} - \mathrm{St}_0 \right) & a < a_{12}\\
        \sqrt{\frac{3}{\mathrm{St} + \mathrm{St}^{-1}}} & a \geq a_{12}
    \end{cases}
\end{align}

\begin{align}
v_\mathrm{eq} = \sqrt{\frac{2}{3}} \,
    \begin{cases}
        0 & a < a_{12}\\
        v_\mathrm{mon} & a \geq a_{12}
    \end{cases}
\end{align}

In [ ]:
# simple recipe

simple_v_mon_0 = Re[:, None]**0.25 * (St - St0)
simple_v_mon_1 = np.sqrt(3 / (St + 1 / St))
simple_v_mon = np.minimum(simple_v_mon_0, simple_v_mon_1)
simple_v_mon *= np.sqrt(3. / 2. * s.gas.alpha[:, None]) * s.gas.cs[:, None]

simple_v_eq = np.sqrt(2 / 3) * simple_v_mon
simple_v_eq[a[None, :] < a_12[:, None]] = 0.0

In [ ]:
import seaborn as sns

In [ ]:
sns.set_palette('Set1')

In [ ]:
f, ax = plt.subplots(dpi=200)

# by how much do we have to shift indices to get double/half the particle size
i_offset = np.abs(a - 2 * a[0]).argmin()

# full ormel formula
ax.loglog(a, np.diag(s.dust.v.rel.turb[ir, :, :]), 'k-')
ax.loglog(a, s.dust.v.rel.turb[ir, :, 0], 'k--')
ax.plot([], 'k-',  label='turbulence, Ormel et al. 2007')

# other contributions
ax.loglog(a[i_offset:], np.diag(s.dust.v.rel.rad[ir, ...], k=i_offset), 'C0-', label='radial drift')
ax.loglog(a[i_offset:], np.diag(s.dust.v.rel.vert[ir, ...], k=i_offset), 'C1-', label='vertical settling')
ax.loglog(a[i_offset:], np.diag(s.dust.v.rel.azi[ir, ...], k=i_offset), 'C2-', label='azimuthal drift')
ax.loglog(a, np.diag(s.dust.v.rel.brown[ir, ...], k=0), 'C3-', label='Brownian motion')

# interpolated version after B11

# ax.loglog(a, dv_0[ir, :], 'C0--')
# ax.loglog(a, dv_1[ir, :], 'C0-')
# ax.plot([], 'C0', label='interp. approximation')

# the two parts of this approximation

# simple_v0 = np.sqrt(s.gas.alpha[:, None]) * s.gas.cs[:, None] * np.sqrt(3 / 2) * Re[:, None]**0.25 * (St - St0)
# simple_v1 = np.sqrt(s.gas.alpha[:, None]) * s.gas.cs[:, None] * np.sqrt(3 / 2) * np.sqrt(3 * St)
# ax.loglog(a, simple_v0[ir, :], 'r--', zorder=100)
# ax.loglog(a, simple_v1[ir, :], 'r--')


# brownian re-computed
# m = s.grid.m
# v_BM = np.sqrt(8 * s.gas.T[ir] * c.k_B * 2 / (np.pi * m))
# ax.loglog(a, v_BM, 'r--', zorder=100)

# simple formula

ax.loglog(a, simple_v_eq[ir, :], c='0.5')
ax.loglog(a, simple_v_mon[ir, :], c='0.5', ls='--')
ax.plot([], c='0.5', label='turbulence, approximation')

# axes styling

ax.set_xlim(1e-5, a[St[ir].searchsorted(10)])
ax.set_ylim(1e-2, 1e4)
ax.set_xlabel('particle size [cm]')
ax.set_ylabel(r'relative velocity [cm / s]')

# secondary x-axis

ax2 = ax.secondary_xaxis('top', functions=(a2St, St2a))
ax2.set_xlabel('Stokes number')

# vertical lines with text

ax.axvline(a_12[ir], c='k', ls='--', lw=.5)
ax.axvline(a_BT[ir], c='k', ls='--', lw=.5)
ax.text(a_12[ir], 4e3, '$a_{12}$', rotation=90, ha='right')
ax.text(a_BT[ir], 4e3, '$a_{BT}$', rotation=90, ha='right')

# legend

ax.plot([], 'k--', label='monomer collisions')
ax.plot([], 'k-',  label='equal-sized collisions')
fr = ax.legend(loc=(0, .55), fontsize='x-small', ).get_frame()
fr.set_facecolor('w');
fr.set_edgecolor('none')
fr.set_alpha(0.9)

In [ ]:
f.savefig(helper.output_dir / 'rel_vel2.pdf', transparent=True, bbox_inches='tight')